# Import de Dependências

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import pyspark.sql.functions as sf
from pyspark.sql.types import IntegerType
from feature_store import FeatureStore, Catalog
from pyspark.sql import DataFrame, Column, DataFrame, SparkSession
import time
import pyspark.sql.functions as sf


import gcsfs
import os
import pyarrow.parquet as pq

from pyspark.sql.functions import (
    col, count, mean, monotonically_increasing_id, row_number, sum, when, lit
)
from pyspark.sql.types import IntegerType, StringType
from functools import reduce
import builtins

In [ ]:
import pyspark
from typing import Callable, Dict, List, Tuple
from pyspark.sql import Column

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)

# Pull do dicionário

In [ ]:
def read_parquet_to_pandas(path: str, projectGCS="analytics-k8s-dev-4742"):
    """function to read parquet file as
     as pandas dataframe - useful for
     training tasks"""
    fs = gcsfs.GCSFileSystem(project=projectGCS, requester_pays=True)
    files = ["gs://" + path for path in fs.glob(path + "/part-*")]
    df_pandas = pq.ParquetDataset(files, filesystem=fs).read().to_pandas()
    return df_pandas

In [ ]:
flags_weights = read_parquet_to_pandas("gs://oculto/loss_prevention/perfil_laranja/data/df_importance_baixa_renda.parquet")

In [ ]:
flags_weights

,Feature,Importance
0,flagAreaUrbanaBaixaDensidadeEdificios,100.000000
1,flagAreaRural,81.858578
2,flagSocioClasseEmpresaFachadaALTA,79.036537
3,flagDividaDauCresceu180Dias,77.803796
4,evolucaoPatrimonioSEM_INFORMACAO,69.294574
...,...,...
120,flagDuplaIdentidadeNomeMae,0.000000
121,flagBaixaRenda,0.000000
122,flagRendaSemInformacao,0.000000
123,flagAltaRenda,0.000000


In [ ]:
flags_weights_dict = flags_weights.set_index('Feature')['Importance'].to_dict()

print(flags_weights_dict)

{'flagAreaUrbanaBaixaDensidadeEdificios': 100.0, 'flagAreaRural': 81.85857849682795, 'flagSocioClasseEmpresaFachadaALTA': 79.03653721889533, 'flagDividaDauCresceu180Dias': 77.80379563553423, 'evolucaoPatrimonioSEM_INFORMACAO': 69.2945741888192, 'flagProdutorRural': 66.84115664000808, 'flagAlterouBancoDeclaracaoIrpfUltimos5Anos': 66.0542185482743, 'flagAltaQualificacao': 59.25441349876753, 'evolucaoPatrimonioDIMINUIU': 55.79424436344653, 'perfilInvestidorDIVERSIFICADO': 53.904411672956876, 'flagBeneficiarioBPC': 49.76119569388573, 'flagSocioBeneficiarioProgramaSocial': 48.865527649992636, 'flagSocioEmpresasGrandePorte': 45.83229775036054, 'flagBeneficiarioNovoBolsaFamilia': 45.50839414327287, 'flagSocioClasseEmpresaFachadaMUITOALTA': 43.7212323037517, 'perfilInvestidorSEM_INFORMACAO': 42.02465334845129, 'quantidadeMediaEmpresasMesmoCnaeAbertas12Meses': 41.60073716519834, 'flagSegundoGrauProcessoJudicialCobranca': 40.26808617605308, 'flagFavelaOuComunidadeUrbana': 37.62385230438633, 'qua

In [ ]:
WEIGHTS_FEATURES = {'flagAreaUrbanaBaixaDensidadeEdificios': 100.0, 'flagAreaRural': 81.67728736599034, 'flagSocioClasseEmpresaFachadaALTA': 78.9071928947527, 'flagDividaDauCresceu180Dias': 77.65217548333106, 'evolucaoPatrimonioSEM_INFORMACAO': 69.09522087580325, 'flagProdutorRural': 66.74385526827432, 'flagAlterouBancoDeclaracaoIrpfUltimos5Anos': 65.83838685935214, 'flagAltaQualificacao': 59.00555602060086, 'evolucaoPatrimonioDIMINUIU': 55.62372890464312, 'perfilInvestidorDIVERSIFICADO': 53.8243192232701, 'flagBeneficiarioBPC': 49.708674544908376, 'flagSocioBeneficiarioProgramaSocial': 48.71561314307856, 'flagSocioEmpresasGrandePorte': 45.80426681077026, 'flagBeneficiarioNovoBolsaFamilia': 45.30383297836726, 'flagSocioClasseEmpresaFachadaMUITOALTA': 43.62452184292625, 'perfilInvestidorSEM_INFORMACAO': 41.90369281717729, 'quantidadeMediaEmpresasMesmoCnaeAbertas12Meses': 41.55684551989452, 'flagSegundoGrauProcessoJudicialCobranca': 40.16395437945208, 'flagFavelaOuComunidadeUrbana': 37.54100031169861, 'quantidadeEleicoesComoDoador': 37.452619000381034, 'flagAltaQualificacaoBeneficiarioProgramasSociais': 36.77578071374251, 'flagSocioClasseEmpresaFachadaMEDIO': 34.51696156130973, 'flagBeneficiarioProgramasSociais': 34.249837086633676, 'flagRegistroAntt': 33.233565331596175, 'flagProcessoJudicialCobranca': 32.97639136697473, 'quantidadeTelefones': 32.33957056230212, 'diversificacaoBest': 31.40930020445709, 'diversificacaoFaixa': 30.70984448632838, 'historicoAtrasoFinanciamentoVeicularNAO_ATRASA': 30.403457211521577, 'historicoAtrasoFinanciamentoVeicularATRASA': 29.94014734181799, 'perfilInvestidorCONSERVADOR': 29.68248081740823, 'qtdImoveis': 28.99415934221374, 'diversificacao': 28.0472612274349, 'flagPrimeiroGrauProcessoJudicialCobranca': 27.099876911711966, 'quantidadeEmailPadraoSuspeito': 26.353991034529844, 'flagTop10ValorImoveisRenda': 25.32610536914729, 'historicoAtrasoFinanciamentoVeicularSEM_INFORMACAO': 25.05212834998172, 'flagDuplaIdentidadeNomeDataNascimento': 23.439386977536444, 'quantidadeDoacoesDescricaoNaoEspecificada': 22.50517236054731, 'bancarizacaoBest': 20.662207210480684, 'valorTotalDoado': 20.561868539276123, 'quantidadePropriedades': 19.934898370454828, 'flagSocioProcessoJudicialTributario': 19.816999807810117, 'ativosSemCota': 19.259425168699167, 'flagIdoso': 19.256910052296984, 'valorMedioDoado': 19.009956856848596, 'ativosComCotaPercentual': 18.873156567927943, 'perfilInvestidorAGRESSIVO': 16.519759120155232, 'quantidadeDeclaracoesIrpfObservadas': 16.409194389364117, 'flagEmpresario': 16.30875574130877, 'RecorrenciaFinanciamentoMaioridade': 16.29254306318486, 'quantidadeEmails': 16.114271158519465, 'bancarizacaoFaixa': 15.936752608505195, 'quantidadeTelefonesEnderecoConsistente': 15.823089603227942, 'ativosComCota': 15.251640085917453, 'flagTop5ValorImoveisRenda': 15.166960655325616, 'quantidadeRestituicaoIrpfObservadas': 14.184586738774547, 'valorTotal': 13.838917399005613, 'quantidadePropriedadesAtivas': 13.196841401808973, 'flagSocioEmpresasEPP': 12.826275680255725, 'qtdFinanciamentosVeicularesQuitados': 12.67148533109892, 'quantidadeDoacoesPoliticas': 12.599424677262528, 'recorrenciaFinanciamentoVeicular': 12.586827465679773, 'quantidadeEmailNaoConsistenteNomeTitular': 12.287420797246885, 'qtdVeiculosFinanciados': 11.616366942472304, 'evolucaoPatrimonioAUMENTOU': 10.36496622533303, 'valorMaximoDoado': 9.535278076139388, 'flagSegundoGrauProcessoJudicialRouboFurtos': 9.315229887036624, 'quantidadeCandidatosApoiados': 9.282542483288921, 'flagBeneficiarioSeguroDefeso': 9.125390796358309, 'quantidadeVeiculosPesados': 8.79313507430477, 'quantidadeDuplaIdentidadeNomeDataNascimento': 8.763799913393413, 'valorAreaTotalPropriedadesRurais': 8.616419272829694, 'patrimonio': 8.545154414822171, 'bancarizacao': 8.308573984480287, 'quantidadeDiferentesprovedoresEmail': 7.825681648142605, 'quantidadeVeiculosPesadosAnoFabricacaoEntre10E14AnosAtras': 6.645988781198244, 'evolucaoPatrimonioMANTEVE': 6.091359120088005, 'valorTotalDividasDau': 5.5759996291556115, 'flagServidorPublico': 4.551307400317049, 'flagBeneficiarioGarantiaSafra': 4.343970431295563, 'flagProcessoJudicialCorrupcao': 3.792414517243508, 'flagSocioProcessoJudicialCorrupcao': 3.3652641675432324, 'flagProcessoJudicialRouboFurtos': 3.2821439392874097, 'flagSocioProcessoJudicialFraude': 2.3885984743285062, 'flagTop10AltoValorImoveisBeneficiario': 2.17469056568461, 'flagProcessoJudicialFraude': 1.3520519540908782, 'flagSocioDuplaId': 1.027452343888671, 'flagFinanciamentoImobiliarioAltoValorRenda': 0.9176985880073287, 'indicioFalecimento': 0.8496656554080556, 'flagSegundoGrauProcessoJudicialCorrupcao': 0.6614668711659378, 'flagAglomeradoRural': 0.33402446522360923, 'flagSegundoGrauProcessoJudicialFraude': 0.2870669689920855, 'flagTop1ValorImoveisRenda': 0.227841253904505, 'flagPrimeiroGrauProcessoJudicialRouboFurtos': 0.15092151939894408, 'flagSocioProcessoJudicialLavagemDinheiro': 0.09087394966573005, 'flagPrimeiroGrauProcessoJudicialCorrupcao': 0.0871583703591381, 'flagProcessoJudicialLavagem': 0.07091430353654438, 'flagPrimeiroGrauProcessoJudicialFraude': 0.055745182205561315, 'flagTop5AltoValorImoveisBeneficiario': 0.016097854885498798, 'flagSegundoGrauProcessoJudicialLavagem': 0.015485970638600107, 'flagVulneravelBPC': 0.014132220931090275, 'quantidadeVeiculosPesadosAnoFabricacaoEntre0E4AnosAtras': 0.013062389392493344, 'quantidadeDuplaIdentidadeNomeMae': 0.004845146979809202, 'flagLaranja': 0.0014496329992596453, 'flagPrimeiroGrauProcessoJudicialLavagem': 3.7802259940606754e-05, 'flagFalecidoConfirmado': 0.0, 'flagSupercentenario': 0.0, 'flagContratoPublicoBeneficiarioProgramasSociais': 0.0, 'quantidadeDuplaIdentidadeCpfMae': 0.0, 'flagAdolescente': 0.0, 'flagFinanciamentoImobiliarioAltoValorBeneficiario': 0.0, 'flagOperadorAeronave': 0.0, 'flagIndicioForteDuplaIdentidade': 0.0, 'quantidadeIndicioForteDuplaIdentidade': 0.0, 'flagDuplaIdentidadeCpfMae': 0.0, 'flagTop1AltoValorImoveisBeneficiario': 0.0, 'flagProprietarioAeronave': 0.0, 'quantidadeAeronaves': 0.0, 'quantidadeVeiculosPesadosAnoFabricacaoEntre5E9AnosAtras': 0.0, 'flagDuplaIdentidadeNomeMae': 0.0, 'flagBaixaRenda': 0.0, 'flagRendaSemInformacao': 0.0, 'flagAltaRenda': 0.0, 'flagMediaRenda': 0.0}

In [ ]:
WEIGHTS_FEATURES

{'flagAreaUrbanaBaixaDensidadeEdificios': 100.0,
 'flagAreaRural': 81.67728736599034,
 'flagSocioClasseEmpresaFachadaALTA': 78.9071928947527,
 'flagDividaDauCresceu180Dias': 77.65217548333106,
 'evolucaoPatrimonioSEM_INFORMACAO': 69.09522087580325,
 'flagProdutorRural': 66.74385526827432,
 'flagAlterouBancoDeclaracaoIrpfUltimos5Anos': 65.83838685935214,
 'flagAltaQualificacao': 59.00555602060086,
 'evolucaoPatrimonioDIMINUIU': 55.62372890464312,
 'perfilInvestidorDIVERSIFICADO': 53.8243192232701,
 'flagBeneficiarioBPC': 49.708674544908376,
 'flagSocioBeneficiarioProgramaSocial': 48.71561314307856,
 'flagSocioEmpresasGrandePorte': 45.80426681077026,
 'flagBeneficiarioNovoBolsaFamilia': 45.30383297836726,
 'flagSocioClasseEmpresaFachadaMUITOALTA': 43.62452184292625,
 'perfilInvestidorSEM_INFORMACAO': 41.90369281717729,
 'quantidadeMediaEmpresasMesmoCnaeAbertas12Meses': 41.55684551989452,
 'flagSegundoGrauProcessoJudicialCobranca': 40.16395437945208,
 'flagFavelaOuComunidadeUrbana': 37.54

# Indicador composto - Modelo da Manu

In [ ]:
class Train:

    def __init__(self, data: pyspark.sql.DataFrame) -> None:
        self._data = data

    def _calculate_score(self, columns_weight: Dict) -> Column:
            """
            Calculates the score based on the columns of a DataFrame and their respective weights.

            Args:
                columns_weight (dict): A dictionary where keys are column names and values
                                       are the corresponding weights for those columns.

            Returns:
                float: The score calculated based on the provided columns and weights.
            """
            #return sum(sf.col(column) * columns_weight[column] for column in columns_weight)
            return builtins.sum([sf.col(col) * weight for col, weight in columns_weight.items()])


    def _calculate_log_columns(
        self,
        df: pyspark.sql.DataFrame,
        columns: List[str]
    ) -> pyspark.sql.DataFrame:
        """
        Applies log1p transformation to a list of columns and renames them.

        Parameters:
        df (pyspark.sql.DataFrame): The input DataFrame.
        columns (list): List of column names to transform.

        Returns:
        pyspark.sql.DataFrame: DataFrame with new columns containing log1p transformations.
        """

        log_columns = [
            sf.log1p(sf.col(col)).alias(f"log{col[0].upper() + col[1:]}")
            for col in columns
        ]

        return df.select("*", *log_columns)

    def _calculate_percentil(
        self,
        df: pyspark.sql.DataFrame,
        column: str,
        percentil: float
    ) -> float:
        """
        Calculate the approximate percentile of a column in a DataFrame.
        """
        return (
            df
            .selectExpr(f"approx_percentile({column}, {percentil}) as percentile")
            .collect()[0]["percentile"]
        )

    def _calculate_normalization_threshold(
        self,
        df: pyspark.sql.DataFrame,
        column_to_normalize: str,
        percentil_dict: Dict[str, float]
    ) -> Dict[str, float]:

        """
        Calculate normalization thresholds (min and max) based on percentiles for a specific column.
        """

        min_threshold_point = self._calculate_percentil(
            df,
            column_to_normalize,
            percentil_dict["min"]
        )

        max_threshold_point = self._calculate_percentil(
            df,
            column_to_normalize,
            percentil_dict["max"]
        )

        return {
            "min": min_threshold_point,
            "max": max_threshold_point
        }

    def _compute_normalization_thresholds_dict(
        self,
        df: pyspark.sql.DataFrame,
        percentiles_dict: Dict[str, Dict[str, float]]
    ) -> Dict[str, Dict[str, float]]:

        """
        Computes normalization thresholds for specified columns in a PySpark DataFrame.

        This function iterates over the given dictionary of percentiles and calculates
        the normalization thresholds for each column using the `_calculate_normalization_threshold` method.

        Args:
            df (pyspark.sql.DataFrame): The input PySpark DataFrame containing the data.
            percentiles_dict (Dict[str, Dict[str, float]]): A dictionary where keys are column names,
                and values are dictionaries containing percentile values used to compute thresholds.

        Returns:
            Dict[str, Dict[str, float]]: A dictionary mapping each column name to its computed
                normalization threshold values.
        """

        thresholds = {}
        for column, percentils in percentiles_dict.items():
            thresholds[column] = self._calculate_normalization_threshold(
                df, column, percentils
            )

        return thresholds

    def _compute_thresholds_dict(
        self,
        df: pyspark.sql.DataFrame,
        percentiles_dict: Dict[str, Dict[str, float]],
        threshold_fn: Callable[[pyspark.sql.DataFrame, str, Dict[str, float]], Dict[str, float]]
    ) -> Dict[str, Dict[str, float]]:

        """
        Computes thresholds for specified columns in a PySpark DataFrame using a given threshold function.

        This function iterates over the provided dictionary of percentiles and applies the given
        `threshold_fn` to compute thresholds for each column.

        Args:
            df (pyspark.sql.DataFrame): The input PySpark DataFrame containing the data.
            percentiles_dict (Dict[str, Dict[str, float]]): A dictionary where keys are column names,
                and values are dictionaries containing percentile values used to compute thresholds.
            threshold_fn (Callable[[pyspark.sql.DataFrame, str, Dict[str, float]], Dict[str, float]]):
                A function that takes a DataFrame, column name, and percentiles dictionary as input
                and returns computed threshold values for that column.

        Returns:
            Dict[str, Dict[str, float]]: A dictionary mapping each column name to its computed
                threshold values.
        """

        thresholds = {}
        for column, percentils in percentiles_dict.items():
            thresholds[column] = threshold_fn(df, column, percentils)

        return thresholds

    def _normalizer(
        self,
        column: str,
        maxThreshold: float,
        minThreshold: float,
        minScore: float = 0.0,
        maxScore: float = 1.0
    ) -> Column:

        """
        Performs Min-Max scaling normalization on a given column of a DataFrame.

        Args:
            column (str): The name of the column to be normalized.
            minThreshold (float): The lower threshold value for normalization.
            maxThreshold (float): The upper threshold value for normalization.

        Returns:
            Column: A PySpark Column containing normalized values based on Min-Max scaling.
        """

        scale = (maxScore - minScore)
        interval_size = (sf.lit(maxThreshold) - sf.lit(minThreshold))

        min_max = scale * ((sf.col(column) - sf.lit(minThreshold)) / interval_size) + minScore

        return sf.when(
            sf.col(column) < sf.lit(maxThreshold),
            sf.when(sf.col(column) > sf.lit(minThreshold), min_max).otherwise(minScore)
        ).otherwise(maxScore)

    def _normalize_column(
        self,
        df: pyspark.sql.DataFrame,
        column_to_normalize: str,
        normalized_column: str,
        thresholds: Dict[str, float]
    ) -> pyspark.sql.DataFrame:

        """
        Normalizes a specified column in a DataFrame by applying percentile-based thresholds. The values in the
        column are scaled between the calculated minimum and maximum threshold points.

        Args:
            df (pyspark.sql.DataFrame): The input DataFrame containing the column to normalize.
            column_to_normalize (str): The name of the column to be normalized.
            normalized_column (str): The name of the output column to store the normalized values.
            thresholds (Dict[str, float]): A dictionary specifying the minimum and maximum percentile values
                for normalization. The dictionary should have the following keys:
                - 'min': The minimum percentile value (e.g., 0.05 for the 5th percentile).
                - 'max': The maximum percentile value (e.g., 0.95 for the 95th percentile).

        Returns:
            pyspark.sql.DataFrame: A DataFrame with an additional column containing the
            normalized values based on the specified percentile thresholds.

        Raises:
            KeyError: If 'min' or 'max' are not present in the `thresholds` dictionary.
            ValueError: If the column to normalize doesn't exist in the DataFrame.
        """

        return df.withColumn(
            normalized_column,
            self._normalizer(column_to_normalize, thresholds["max"], thresholds["min"])
        )

    def _calculate_categorization_threshold(
        self,
        df: pyspark.sql.DataFrame,
        column_to_categorize: str,
        percentil_dict: Dict[str, float]
    ) -> Dict[str, float]:

        """
        Calculate classification thresholds (low, medium, high) based on
        percentiles for a specific column.
        """

        low_threshold_point = self._calculate_percentil(df, column_to_categorize, percentil_dict["low"])
        medium_threshold_point = self._calculate_percentil(df, column_to_categorize, percentil_dict["medium"])
        high_threshold_point = self._calculate_percentil(df, column_to_categorize, percentil_dict["high"])

        return {
            "low": low_threshold_point,
            "medium": medium_threshold_point,
            "high": high_threshold_point
        }

    def fit(self) -> Tuple[
        pyspark.sql.DataFrame,
        Dict[str, Dict[str, float]],
        Dict[str, Dict[str, float]]]:

        """
        Main function that trains the model by calculating both normalization
        and classification thresholds for the provided data and returns them
        in a dictionary format.
        """

        core_columns_percentiles = {
            "logLeadPoints": {"min": 0.0, "max": 0.99}
        }

        data_scores = (
            self._data
            .withColumn("leadPoints", self._calculate_score(WEIGHTS_FEATURES))
        )

        data_scores = self._calculate_log_columns(data_scores, ["leadPoints"])

        data_scores_not_null = data_scores.filter(sf.col("logLeadPoints").isNotNull())

        normalization_thresholds = self._compute_thresholds_dict(
            data_scores_not_null,
            core_columns_percentiles,
            self._calculate_normalization_threshold
        )

        core_columns = {
            "logLeadPoints": "leadScore",
        }

        for column_to_normalize, normalized_column in core_columns.items():
            data_scores_not_null = self._normalize_column(
                data_scores_not_null,
                column_to_normalize,
                normalized_column,
                normalization_thresholds[column_to_normalize]
            )

        categorization_percentiles = {
            "leadScore": {
                "low": 0.50,
                "medium": 0.75,
                "high": 0.9
            }
        }

        categorization_thresholds = self._compute_thresholds_dict(
            data_scores_not_null,
            categorization_percentiles,
            self._calculate_categorization_threshold
        )

        return data_scores_not_null, normalization_thresholds, categorization_thresholds


# Execução do Indicador

In [ ]:
teste = spark.read.parquet("gs://oculto/loss_prevention/perfil_laranja/data/perfil_laranja_clean_baixaRenda_reduzido.parquet")
teste = teste.sample(fraction=0.1, seed=2025)

In [ ]:
teste.count()

477889

In [ ]:
%%time
train_model = Train(teste)

CPU times: user 8 μs, sys: 0 ns, total: 8 μs
Wall time: 15.7 μs


In [ ]:
print(type(teste))

<class 'pyspark.sql.dataframe.DataFrame'>


A Manu usa uma função em Main, mas já estou fazendo o que essa função faz ao restante do codigo

In [ ]:
df_resultado, normalization_thresholds, categorization_thresholds = train_model.fit()

In [ ]:
df_resultado.limit(5).toPandas()


,flagBeneficiarioBPC,flagVulneravelBPC,flagBeneficiarioSeguroDefeso,flagBeneficiarioGarantiaSafra,flagBeneficiarioNovoBolsaFamilia,flagBeneficiarioProgramasSociais,flagAltaQualificacaoBeneficiarioProgramasSociais,indicioFalecimento,flagFalecidoConfirmado,flagSupercentenario,flagAdolescente,flagIdoso,flagEmpresario,flagAltaQualificacao,flagServidorPublico,flagRegistroAntt,flagProdutorRural,quantidadeCandidatosApoiados,quantidadeEleicoesComoDoador,valorMedioDoado,valorTotalDoado,quantidadeDoacoesDescricaoNaoEspecificada,valorMaximoDoado,quantidadeDoacoesPoliticas,flagContratoPublicoBeneficiarioProgramasSociais,flagLaranja,quantidadeEmails,quantidadeDiferentesprovedoresEmail,quantidadeEmailPadraoSuspeito,quantidadeEmailNaoConsistenteNomeTitular,flagAreaUrbanaBaixaDensidadeEdificios,flagAglomeradoRural,flagAreaRural,flagFavelaOuComunidadeUrbana,flagDividaDauCresceu180Dias,valorTotalDividasDau,qtdVeiculosFinanciados,qtdFinanciamentosVeicularesQuitados,recorrenciaFinanciamentoVeicular,RecorrenciaFinanciamentoMaioridade,flagFinanciamentoImobiliarioAltoValorBeneficiario,flagFinanciamentoImobiliarioAltoValorRenda,quantidadeRestituicaoIrpfObservadas,quantidadeDeclaracoesIrpfObservadas,flagAlterouBancoDeclaracaoIrpfUltimos5Anos,ativosComCota,ativosComCotaPercentual,ativosSemCota,bancarizacao,bancarizacaoBest,bancarizacaoFaixa,diversificacao,diversificacaoBest,diversificacaoFaixa,patrimonio,flagSocioProcessoJudicialLavagemDinheiro,flagProcessoJudicialCorrupcao,flagProcessoJudicialLavagem,flagProcessoJudicialFraude,flagProcessoJudicialRouboFurtos,flagSocioProcessoJudicialFraude,flagProcessoJudicialCobranca,flagSocioProcessoJudicialCorrupcao,flagSocioProcessoJudicialTributario,quantidadeDuplaIdentidadeNomeDataNascimento,flagDuplaIdentidadeNomeDataNascimento,quantidadeDuplaIdentidadeNomeMae,flagDuplaIdentidadeNomeMae,quantidadeDuplaIdentidadeCpfMae,flagDuplaIdentidadeCpfMae,quantidadeIndicioForteDuplaIdentidade,flagIndicioForteDuplaIdentidade,qtdImoveis,valorTotal,flagTop10ValorImoveisRenda,flagTop5ValorImoveisRenda,flagTop1ValorImoveisRenda,flagTop10AltoValorImoveisBeneficiario,flagTop5AltoValorImoveisBeneficiario,flagTop1AltoValorImoveisBeneficiario,quantidadeVeiculosPesados,quantidadeVeiculosPesadosAnoFabricacaoEntre0E4AnosAtras,quantidadeVeiculosPesadosAnoFabricacaoEntre5E9AnosAtras,quantidadeVeiculosPesadosAnoFabricacaoEntre10E14AnosAtras,valorAreaTotalPropriedadesRurais,quantidadePropriedades,quantidadePropriedadesAtivas,flagOperadorAeronave,flagProprietarioAeronave,quantidadeAeronaves,flagPrimeiroGrauProcessoJudicialCorrupcao,flagPrimeiroGrauProcessoJudicialLavagem,flagPrimeiroGrauProcessoJudicialFraude,flagPrimeiroGrauProcessoJudicialRouboFurtos,flagPrimeiroGrauProcessoJudicialCobranca,flagSegundoGrauProcessoJudicialCorrupcao,flagSegundoGrauProcessoJudicialLavagem,flagSegundoGrauProcessoJudicialFraude,flagSegundoGrauProcessoJudicialRouboFurtos,flagSegundoGrauProcessoJudicialCobranca,flagRendaSemInformacao,flagBaixaRenda,flagMediaRenda,flagAltaRenda,flagSocioDuplaId,quantidadeMediaEmpresasMesmoCnaeAbertas12Meses,flagSocioEmpresasGrandePorte,flagSocioEmpresasEPP,flagSocioBeneficiarioProgramaSocial,flagSocioClasseEmpresaFachadaALTA,flagSocioClasseEmpresaFachadaMUITOALTA,flagSocioClasseEmpresaFachadaMEDIO,quantidadeTelefones,quantidadeTelefonesEnderecoConsistente,historicoAtrasoFinanciamentoVeicularSEM_INFORMACAO,historicoAtrasoFinanciamentoVeicularATRASA,historicoAtrasoFinanciamentoVeicularNAO_ATRASA,evolucaoPatrimonioSEM_INFORMACAO,evolucaoPatrimonioAUMENTOU,evolucaoPatrimonioDIMINUIU,evolucaoPatrimonioMANTEVE,perfilInvestidorSEM_INFORMACAO,perfilInvestidorDIVERSIFICADO,perfilInvestidorCONSERVADOR,perfilInvestidorAGRESSIVO,leadPoints,logLeadPoints,leadScore
0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,525.0,525.0,1,525.0,1,0,0,0,0,0,0,0,0,1,0,0,0.00,2,2,7,13.0,0,0,0,4,0,-1,0,-1,-1,-1,-1,-1,-1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0.0,1,0,0,0,0,0,2,2,0.0,0.0,1.0,1.0,0.0,

# Salvar o indicador

In [ ]:
df_resultado.write.mode("overwrite").parquet(
    "gs://oculto/loss_prevention/perfil_laranja/data/indicador_composto_baixa_renda.parquet"
)

## Edna - Neoway Soluções

In [ ]:
dataset = spark.read.parquet("gs://oculto/loss_prevention/perfil_laranja/data/perfil_laranja_clean_com_cpf.parquet")

In [ ]:
dt = dataset.filter(sf.col("cpf") == '55252833877')

In [ ]:
dt.limit(5).toPandas()

,cpf,flagBeneficiarioBPC,flagVulneravelBPC,flagBeneficiarioSeguroDefeso,flagBeneficiarioGarantiaSafra,flagBeneficiarioNovoBolsaFamilia,flagBeneficiarioProgramasSociais,flagAltaQualificacaoBeneficiarioProgramasSociais,indicioFalecimento,flagFalecidoConfirmado,flagSupercentenario,flagAdolescente,flagIdoso,flagEmpresario,flagAltaQualificacao,flagServidorPublico,flagRegistroAntt,flagProdutorRural,quantidadeCandidatosApoiados,quantidadeEleicoesComoDoador,valorMedioDoado,valorTotalDoado,quantidadeDoacoesDescricaoNaoEspecificada,valorMaximoDoado,quantidadeDoacoesPoliticas,flagContratoPublicoBeneficiarioProgramasSociais,flagLaranja,quantidadeEmails,quantidadeDiferentesprovedoresEmail,quantidadeEmailPadraoSuspeito,quantidadeEmailNaoConsistenteNomeTitular,flagAreaUrbanaBaixaDensidadeEdificios,flagAglomeradoRural,flagAreaRural,flagFavelaOuComunidadeUrbana,flagDividaDauCresceu180Dias,valorTotalDividasDau,qtdVeiculosFinanciados,qtdFinanciamentosVeicularesQuitados,recorrenciaFinanciamentoVeicular,RecorrenciaFinanciamentoMaioridade,flagFinanciamentoImobiliarioAltoValorBeneficiario,flagFinanciamentoImobiliarioAltoValorRenda,quantidadeRestituicaoIrpfObservadas,quantidadeDeclaracoesIrpfObservadas,flagAlterouBancoDeclaracaoIrpfUltimos5Anos,ativosComCota,ativosComCotaPercentual,ativosSemCota,bancarizacao,bancarizacaoBest,bancarizacaoFaixa,diversificacao,diversificacaoBest,diversificacaoFaixa,patrimonio,flagSocioProcessoJudicialLavagemDinheiro,flagProcessoJudicialCorrupcao,flagProcessoJudicialLavagem,flagProcessoJudicialFraude,flagProcessoJudicialRouboFurtos,flagSocioProcessoJudicialFraude,flagProcessoJudicialCobranca,flagSocioProcessoJudicialCorrupcao,flagSocioProcessoJudicialTributario,quantidadeDuplaIdentidadeNomeDataNascimento,flagDuplaIdentidadeNomeDataNascimento,quantidadeDuplaIdentidadeNomeMae,flagDuplaIdentidadeNomeMae,quantidadeDuplaIdentidadeCpfMae,flagDuplaIdentidadeCpfMae,quantidadeIndicioForteDuplaIdentidade,flagIndicioForteDuplaIdentidade,qtdImoveis,valorTotal,flagTop10ValorImoveisRenda,flagTop5ValorImoveisRenda,flagTop1ValorImoveisRenda,flagTop10AltoValorImoveisBeneficiario,flagTop5AltoValorImoveisBeneficiario,flagTop1AltoValorImoveisBeneficiario,quantidadeVeiculosPesados,quantidadeVeiculosPesadosAnoFabricacaoEntre0E4AnosAtras,quantidadeVeiculosPesadosAnoFabricacaoEntre5E9AnosAtras,quantidadeVeiculosPesadosAnoFabricacaoEntre10E14AnosAtras,valorAreaTotalPropriedadesRurais,quantidadePropriedades,quantidadePropriedadesAtivas,flagOperadorAeronave,flagProprietarioAeronave,quantidadeAeronaves,flagPrimeiroGrauProcessoJudicialCorrupcao,flagPrimeiroGrauProcessoJudicialLavagem,flagPrimeiroGrauProcessoJudicialFraude,flagPrimeiroGrauProcessoJudicialRouboFurtos,flagPrimeiroGrauProcessoJudicialCobranca,flagSegundoGrauProcessoJudicialCorrupcao,flagSegundoGrauProcessoJudicialLavagem,flagSegundoGrauProcessoJudicialFraude,flagSegundoGrauProcessoJudicialRouboFurtos,flagSegundoGrauProcessoJudicialCobranca,flagRendaSemInformacao,flagBaixaRenda,flagMediaRenda,flagAltaRenda,flagSocioDuplaId,quantidadeMediaEmpresasMesmoCnaeAbertas12Meses,flagSocioEmpresasGrandePorte,flagSocioEmpresasEPP,flagSocioBeneficiarioProgramaSocial,flagSocioClasseEmpresaFachadaALTA,flagSocioClasseEmpresaFachadaMUITOALTA,flagSocioClasseEmpresaFachadaMEDIO,quantidadeTelefones,quantidadeTelefonesEnderecoConsistente,historicoAtrasoFinanciamentoVeicularSEM_INFORMACAO,historicoAtrasoFinanciamentoVeicularATRASA,historicoAtrasoFinanciamentoVeicularNAO_ATRASA,evolucaoPatrimonioSEM_INFORMACAO,evolucaoPatrimonioAUMENTOU,evolucaoPatrimonioDIMINUIU,evolucaoPatrimonioMANTEVE,perfilInvestidorSEM_INFORMACAO,perfilInvestidorDIVERSIFICADO,perfilInvestidorCONSERVADOR,perfilInvestidorAGRESSIVO
0,55252833877,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,1,1,706.0,706.0,0,706.0,1,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,-1,0.0,0,0,0,3,0,-1,0,-1,-1,-1,-1,-1,-1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0.0,0,1,1,0,1,0,0,0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,

In [ ]:
train_model = Train(dt)

In [ ]:
df_resultado, normalization_thresholds, categorization_thresholds = train_model.fit()

In [ ]:
df_resultado.limit(5).toPandas()

,cpf,flagBeneficiarioBPC,flagVulneravelBPC,flagBeneficiarioSeguroDefeso,flagBeneficiarioGarantiaSafra,flagBeneficiarioNovoBolsaFamilia,flagBeneficiarioProgramasSociais,flagAltaQualificacaoBeneficiarioProgramasSociais,indicioFalecimento,flagFalecidoConfirmado,flagSupercentenario,flagAdolescente,flagIdoso,flagEmpresario,flagAltaQualificacao,flagServidorPublico,flagRegistroAntt,flagProdutorRural,quantidadeCandidatosApoiados,quantidadeEleicoesComoDoador,valorMedioDoado,valorTotalDoado,quantidadeDoacoesDescricaoNaoEspecificada,valorMaximoDoado,quantidadeDoacoesPoliticas,flagContratoPublicoBeneficiarioProgramasSociais,flagLaranja,quantidadeEmails,quantidadeDiferentesprovedoresEmail,quantidadeEmailPadraoSuspeito,quantidadeEmailNaoConsistenteNomeTitular,flagAreaUrbanaBaixaDensidadeEdificios,flagAglomeradoRural,flagAreaRural,flagFavelaOuComunidadeUrbana,flagDividaDauCresceu180Dias,valorTotalDividasDau,qtdVeiculosFinanciados,qtdFinanciamentosVeicularesQuitados,recorrenciaFinanciamentoVeicular,RecorrenciaFinanciamentoMaioridade,flagFinanciamentoImobiliarioAltoValorBeneficiario,flagFinanciamentoImobiliarioAltoValorRenda,quantidadeRestituicaoIrpfObservadas,quantidadeDeclaracoesIrpfObservadas,flagAlterouBancoDeclaracaoIrpfUltimos5Anos,ativosComCota,ativosComCotaPercentual,ativosSemCota,bancarizacao,bancarizacaoBest,bancarizacaoFaixa,diversificacao,diversificacaoBest,diversificacaoFaixa,patrimonio,flagSocioProcessoJudicialLavagemDinheiro,flagProcessoJudicialCorrupcao,flagProcessoJudicialLavagem,flagProcessoJudicialFraude,flagProcessoJudicialRouboFurtos,flagSocioProcessoJudicialFraude,flagProcessoJudicialCobranca,flagSocioProcessoJudicialCorrupcao,flagSocioProcessoJudicialTributario,quantidadeDuplaIdentidadeNomeDataNascimento,flagDuplaIdentidadeNomeDataNascimento,quantidadeDuplaIdentidadeNomeMae,flagDuplaIdentidadeNomeMae,quantidadeDuplaIdentidadeCpfMae,flagDuplaIdentidadeCpfMae,quantidadeIndicioForteDuplaIdentidade,flagIndicioForteDuplaIdentidade,qtdImoveis,valorTotal,flagTop10ValorImoveisRenda,flagTop5ValorImoveisRenda,flagTop1ValorImoveisRenda,flagTop10AltoValorImoveisBeneficiario,flagTop5AltoValorImoveisBeneficiario,flagTop1AltoValorImoveisBeneficiario,quantidadeVeiculosPesados,quantidadeVeiculosPesadosAnoFabricacaoEntre0E4AnosAtras,quantidadeVeiculosPesadosAnoFabricacaoEntre5E9AnosAtras,quantidadeVeiculosPesadosAnoFabricacaoEntre10E14AnosAtras,valorAreaTotalPropriedadesRurais,quantidadePropriedades,quantidadePropriedadesAtivas,flagOperadorAeronave,flagProprietarioAeronave,quantidadeAeronaves,flagPrimeiroGrauProcessoJudicialCorrupcao,flagPrimeiroGrauProcessoJudicialLavagem,flagPrimeiroGrauProcessoJudicialFraude,flagPrimeiroGrauProcessoJudicialRouboFurtos,flagPrimeiroGrauProcessoJudicialCobranca,flagSegundoGrauProcessoJudicialCorrupcao,flagSegundoGrauProcessoJudicialLavagem,flagSegundoGrauProcessoJudicialFraude,flagSegundoGrauProcessoJudicialRouboFurtos,flagSegundoGrauProcessoJudicialCobranca,flagRendaSemInformacao,flagBaixaRenda,flagMediaRenda,flagAltaRenda,flagSocioDuplaId,quantidadeMediaEmpresasMesmoCnaeAbertas12Meses,flagSocioEmpresasGrandePorte,flagSocioEmpresasEPP,flagSocioBeneficiarioProgramaSocial,flagSocioClasseEmpresaFachadaALTA,flagSocioClasseEmpresaFachadaMUITOALTA,flagSocioClasseEmpresaFachadaMEDIO,quantidadeTelefones,quantidadeTelefonesEnderecoConsistente,historicoAtrasoFinanciamentoVeicularSEM_INFORMACAO,historicoAtrasoFinanciamentoVeicularATRASA,historicoAtrasoFinanciamentoVeicularNAO_ATRASA,evolucaoPatrimonioSEM_INFORMACAO,evolucaoPatrimonioAUMENTOU,evolucaoPatrimonioDIMINUIU,evolucaoPatrimonioMANTEVE,perfilInvestidorSEM_INFORMACAO,perfilInvestidorDIVERSIFICADO,perfilInvestidorCONSERVADOR,perfilInvestidorAGRESSIVO,leadPoints,logLeadPoints,leadScore
0,55252833877,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,1,1,706.0,706.0,0,706.0,1,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,-1,0.0,0,0,0,3,0,-1,0,-1,-1,-1,-1,-1,-1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0.0,0,1,1,0,1,0,0,0,1.0,0